En el presente código se entrenan y evalúan tres modelos de clasificación de regresión logística, donde como variables independientes se utilizan los valores de diferentes variables meteorológicas para cada uno de los diferentes días anteriores al momento de la predicción. La variable objetivo es el nivel cualitativo de la precipitación del día siguiente en términos de los percentiles de la Climatología, clasificada de la siguiente forma:

Clase -1: Si la lluvia del día siguiente es inferior o igual al P(1/3)

Clase 0: Si la lluvia del día siguiente es mayor al P(1/3) y menor al P(2/3)

Clase +1: Si la lluvia del día siguiente es superior o igual al P(2/3)

Un modelo es entrenado con datos de todo el año, el segundo con datos solo de la época lluviosa, y el tercero con datos de la época seca. 

Se realizan tres validaciones para cada modelo: una con datos de todo el año, una con datos exclusivos de la época lluviosa y la última con datos de la época seca.

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# IMPORTACIÓN Y PREPARACIÓN DE DATOS

In [ ]:
import pandas as pd
import numpy as np

# Se importan los datos de entrenamiento de todo el año
file_path = 'Train_all.xlsx'
trainall = pd.read_excel(file_path)

# Se importan los datos de validación de todo el año
file_path = 'Test_all.xlsx'
testall = pd.read_excel(file_path)

#--------------------------------------------------------------------

# Se importan los datos de entrenamiento de la época lluviosa
file_path = 'Train_rainy.xlsx'
trainrainy = pd.read_excel(file_path)

# Se importan los datos de validación de la época lluviosa
file_path = 'Test_rainy.xlsx'
testrainy = pd.read_excel(file_path)

#--------------------------------------------------------------------

# Se importan los datos de entrenamiento de la época seca
file_path = 'Train_dry.xlsx'
traindry = pd.read_excel(file_path)

# Se importan los datos de validación de la época seca
file_path = 'Test_dry.xlsx'
testdry = pd.read_excel(file_path)

### SEPARACIÓN DE DATOS EN X,Y

In [ ]:
# Separamos los datos de entrenamiento de todo el año en variables independientes (X_train) y variable objetivo (Y_train)
Y_trainall = pd.DataFrame()
Y_trainall['percentil'] = trainall['percentil']
X_trainall = trainall.drop(columns='percentil')

# Separamos los datos de validación de todo el año en variables independientes (X_test) y variable objetivo (Y_test)
Y_testall = pd.DataFrame()
Y_testall['percentil'] = testall['percentil']
X_testall = testall.drop(columns='percentil')

#--------------------------------------------------------------------------------------------------------------------------

# Separamos los datos de entrenamiento de época lluviosa en variables independientes (X_train) y variable objetivo (Y_train)
Y_trainrainy = pd.DataFrame()
Y_trainrainy['percentil'] = trainrainy['percentil']
X_trainrainy = trainrainy.drop(columns='percentil')

# Separamos los datos de validación de la época lluviosa en variables independientes (X_test) y variable objetivo (Y_test)
Y_testrainy = pd.DataFrame()
Y_testrainy['percentil'] = testrainy['percentil']
X_testrainy = testrainy.drop(columns='percentil')

#--------------------------------------------------------------------------------------------------------------------------

# Separamos los datos de entrenamiento de época seca en variables independientes (X_train) y variable objetivo (Y_train)
Y_traindry = pd.DataFrame()
Y_traindry['percentil'] = traindry['percentil']
X_traindry = traindry.drop(columns='percentil')

# Separamos los datos de validación de la época seca en variables independientes (X_test) y variable objetivo (Y_test)
Y_testdry = pd.DataFrame()
Y_testdry['percentil'] = testdry['percentil']
X_testdry = testdry.drop(columns='percentil')

#--------------------------------------------------------------------------------------------------------------------------

### SE ESCALAN LOS DATOS EN X

In [ ]:
from sklearn.preprocessing import StandardScaler

# Escalamos los datos en X de entrenamiento y validación de todo el año
X_trainall_scaled = StandardScaler().fit_transform(X_trainall)
X_testall_scaled = StandardScaler().fit_transform(X_testall)

#--------------------------------------------------------------------------------------------------------------------------

# Escalamos los datos en X de entrenamiento y validación de la época lluviosa
X_trainrainy_scaled = StandardScaler().fit_transform(X_trainrainy)
X_testrainy_scaled = StandardScaler().fit_transform(X_testrainy)

#--------------------------------------------------------------------------------------------------------------------------

# Escalamos los datos en X de entrenamiento y validación de la época seca
X_traindry_scaled = StandardScaler().fit_transform(X_traindry)
X_testdry_scaled = StandardScaler().fit_transform(X_testdry)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ENTRENAMIENTO CON DATOS DE TODO EL AÑO

### ENTRENAMIENTO DEL MODELO

In [ ]:
from sklearn.linear_model import LogisticRegression
import joblib
import time

modelo = LogisticRegression()

start_time = time.time()
modelo.fit(X_trainall_scaled, Y_trainall.values.ravel())
end_time = time.time()

training_time = end_time - start_time

# Guarda el modelo en un archivo
joblib.dump(modelo, 'log_regr_comb1_5days_alltrain.pkl')

print('EL MODELO HA SIDO ENTRENADO EXITOSAMENTE')
print()
print(f'Tiempo de entrenamiento: {training_time:.4f} segundos')

In [ ]:
#Obtenemos la importancia de cada predictor
coeficientes = modelo.coef_
interceptos = modelo.intercept_
clases = modelo.classes_

nombres_columnas = X_trainall.columns

# Imprimir resultados
for i in range(0,len(clases)):
    coeficientes_clase = coeficientes[i]
    intercepto = interceptos[i]
    print()
    print(f'DATOS DEL MODELO PARA LA CLASE {clases[i]}')
    print()
    print('Importancias de las caracteristicas:')
    for nombre, coef in zip(nombres_columnas, coeficientes_clase):
        print(f'{nombre}: {coef}')
    print('Intercepto del modelo:')
    print(intercepto)
    print()
    print('-----------------------------------------------------------')
    print('-----------------------------------------------------------')

### EVALUACIÓN DEL MODELO

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE TODO EL AÑO--------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predall = modelo.predict(X_testall_scaled)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testall, Y_predall)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA LLUVIOSA-----------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predrainy = modelo.predict(X_testrainy_scaled)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testrainy, Y_predrainy)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA SECA---------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_preddry = modelo.predict(X_testdry_scaled)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testdry, Y_preddry)
print("Matriz de Confusión:")
print(conf_matrix)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ENTRENAMIENTO CON DATOS DE LA ÉPOCA LLUVIOSA

### ENTRENAMIENTO DEL MODELO

In [ ]:
from sklearn.linear_model import LogisticRegression
import joblib
import time

modelo = LogisticRegression()

start_time = time.time()
modelo.fit(X_trainrainy_scaled, Y_trainrainy.values.ravel())
end_time = time.time()

training_time = end_time - start_time

# Guarda el modelo en un archivo
joblib.dump(modelo, 'log_regr_comb1_5days_rainytrain.pkl')

print('EL MODELO HA SIDO ENTRENADO EXITOSAMENTE')
print()
print(f'Tiempo de entrenamiento: {training_time:.4f} segundos')

In [ ]:
#Obtenemos la importancia de cada predictor
coeficientes = modelo.coef_
interceptos = modelo.intercept_
clases = modelo.classes_

nombres_columnas = X_trainall.columns

# Imprimir resultados
for i in range(0,len(clases)):
    coeficientes_clase = coeficientes[i]
    intercepto = interceptos[i]
    print()
    print(f'DATOS DEL MODELO PARA LA CLASE {clases[i]}')
    print()
    print('Importancias de las caracteristicas:')
    for nombre, coef in zip(nombres_columnas, coeficientes_clase):
        print(f'{nombre}: {coef}')
    print('Intercepto del modelo:')
    print(intercepto)
    print()
    print('-----------------------------------------------------------')
    print('-----------------------------------------------------------')

### EVALUACIÓN DEL MODELO

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE TODO EL AÑO--------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predall = modelo.predict(X_testall_scaled)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testall, Y_predall)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA LLUVIOSA-----------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predrainy = modelo.predict(X_testrainy_scaled)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testrainy, Y_predrainy)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA SECA---------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_preddry = modelo.predict(X_testdry_scaled)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testdry, Y_preddry)
print("Matriz de Confusión:")
print(conf_matrix)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ENTRENAMIENTO CON DATOS DE LA ÉPOCA SECA

### ENTRENAMIENTO DEL MODELO

In [ ]:
from sklearn.linear_model import LogisticRegression
import joblib
import time

modelo = LogisticRegression()

start_time = time.time()
modelo.fit(X_traindry_scaled, Y_traindry.values.ravel())
end_time = time.time()

training_time = end_time - start_time

# Guarda el modelo en un archivo
joblib.dump(modelo, 'log_regr_comb1_5days_drytrain.pkl')

print('EL MODELO HA SIDO ENTRENADO EXITOSAMENTE')
print()
print(f'Tiempo de entrenamiento: {training_time:.4f} segundos')

In [ ]:
#Obtenemos la importancia de cada predictor
coeficientes = modelo.coef_
interceptos = modelo.intercept_
clases = modelo.classes_

nombres_columnas = X_trainall.columns

# Imprimir resultados
for i in range(0,len(clases)):
    coeficientes_clase = coeficientes[i]
    intercepto = interceptos[i]
    print()
    print(f'DATOS DEL MODELO PARA LA CLASE {clases[i]}')
    print()
    print('Importancias de las caracteristicas:')
    for nombre, coef in zip(nombres_columnas, coeficientes_clase):
        print(f'{nombre}: {coef}')
    print('Intercepto del modelo:')
    print(intercepto)
    print()
    print('-----------------------------------------------------------')
    print('-----------------------------------------------------------')

### EVALUACIÓN DEL MODELO

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE TODO EL AÑO--------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predall = modelo.predict(X_testall_scaled)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testall, Y_predall)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA LLUVIOSA-----------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predrainy = modelo.predict(X_testrainy_scaled)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testrainy, Y_predrainy)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA SECA---------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_preddry = modelo.predict(X_testdry_scaled)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testdry, Y_preddry)
print("Matriz de Confusión:")
print(conf_matrix)